In [119]:
import pandas as pd
from pulp import *

In [120]:
df_distance = pd.read_excel('../../Datos_P1/df_distance_km.xlsx')
df_location = pd.read_excel('../../Datos_P1/df_location.xlsx')
df_orders = pd.read_excel('../../Datos_P1/df_orders.xlsx')
df_vehicle = pd.read_excel('../../Datos_P1/df_vehicle.xlsx')

df_distance.index = df_distance.columns

In [121]:
df_vehiculos_aleatorios = df_vehicle.sample(n=4, random_state=420) #Elección aleatoria de 3 vehiculos del dataframe, con un random_state para que sea más consistente
#df_vehiculos_aleatorios.reset_index(drop=True ,inplace=True) #Resetear el index del nuevo dataframe para que tenga los numeros 0, 1 y 2
df_vehicle = df_vehiculos_aleatorios

# con 3 falla :(

Esto te elige 3 vehiculos aleatorios del dataframe de vehiculos, lo he dejado con un random_state para que sea consistente, aunque deberiamos probarlo sin ello más adelante

In [122]:
# Preparación datos para PuLP
capacidades = dict(zip(df_vehicle["vehiculo_id"], df_vehicle["capacidad_kg"]))
costos = dict(zip(df_vehicle["vehiculo_id"], df_vehicle["costo_km"]))
autonomias = dict(zip(df_vehicle["vehiculo_id"], df_vehicle["autonomia_km"]))

pedidos = dict(zip(df_orders["cliente"], df_orders["order_demand"]))
pedidos["Almacén"] = 0  # Aseguramos que el almacén no tenga demanda

distancias = df_distance.stack().to_dict()  # Convierte a un diccionario (i, j): distancia

clientes = df_orders["cliente"].tolist() + ["Almacén"]  # Incluir el almacén como un cliente especial
vehiculos = df_vehicle["vehiculo_id"].tolist()

In [123]:
# Definir el problema de optimización
problem = LpProblem("Ruteo_de_Vehiculos", LpMinimize)

# Listas de clientes y vehículos
clientes = df_orders["cliente"].tolist()
vehiculos = df_vehicle["vehiculo_id"].tolist()
almacen = "Almacén"

# Crear variables de decisión
x = {(v, i, j): LpVariable(f"x_{v}_{i}_{j}", cat="Binary") 
     for v in vehiculos for i in [almacen] + clientes for j in clientes + [almacen] if i != j and df_distance.at[i, j] > 0}

y = {(v, i): LpVariable(f"y_{v}_{i}", cat="Binary") 
     for v in vehiculos for i in clientes}

# Variables auxiliares para evitar subciclos
u = {i: LpVariable(f"u_{i}", lowBound=0, upBound=len(clientes), cat="Continuous") for i in clientes}

# Función objetivo: minimizar costo total del recorrido
problem += lpSum(x[v, i, j] * df_distance.at[i, j] * df_vehicle[df_vehicle["vehiculo_id"] == v]["costo_km"].values[0] 
                 for v in vehiculos for i in [almacen] + clientes for j in clientes + [almacen] if i != j and df_distance.at[i, j] > 0)

# Restricción: Cada cliente debe ser visitado exactamente una vez
for i in clientes:
    problem += lpSum(y[v, i] for v in vehiculos) == 1

# Restricción: Capacidad del vehículo
for v in vehiculos:
    problem += lpSum(y[v, i] * df_orders[df_orders["cliente"] == i]["order_demand"].values[0] 
                     for i in clientes) <= df_vehicle[df_vehicle["vehiculo_id"] == v]["capacidad_kg"].values[0]

# Restricción: Autonomía del vehículo
for v in vehiculos:
    problem += lpSum(x[v, i, j] * df_distance.at[i, j] 
                     for i in [almacen] + clientes for j in clientes + [almacen] if i != j and df_distance.at[i, j] > 0) <= df_vehicle[df_vehicle["vehiculo_id"] == v]["autonomia_km"].values[0]

# Restricción: Salida y llegada desde el almacén
for v in vehiculos:
    problem += lpSum(x[v, almacen, j] for j in clientes if df_distance.at[almacen, j] > 0) == 1
    problem += lpSum(x[v, i, almacen] for i in clientes if df_distance.at[i, almacen] > 0) == 1

# Restricción: Si un vehículo visita un cliente, debe salir de él
for v in vehiculos:
    for i in clientes:
        problem += lpSum(x[v, i, j] for j in clientes + [almacen] if i != j and df_distance.at[i, j] > 0) == y[v, i]
        problem += lpSum(x[v, j, i] for j in [almacen] + clientes if i != j and df_distance.at[j, i] > 0) == y[v, i]

# Restricción para evitar subciclos
for i in clientes:
    for j in clientes:
        if i != j and df_distance.at[i, j] > 0:
            problem += u[i] - u[j] + (len(clientes) * lpSum(x[v, i, j] for v in vehiculos)) <= len(clientes) - 1

# Resolver el problema con un límite de tiempo
problem.solve(PULP_CBC_CMD(timeLimit=60))

1

# Restricción: Un vehículo debe volver al almacén antes de sobrepasar su capacidad
for v in vehiculos:
    for i in clientes:
        problem += lpSum(x[v, i, j] for j in clientes + [almacen] if (v, i, j) in x) - lpSum(x[v, almacen, j] for j in clientes if (v, almacen, j) in x) <= 0

In [124]:
# Calcular costos, peso total y capacidad utilizada
costo_total = 0
print("\nResumen de rutas:\n" + "="*40)
for v in vehiculos:
    costo_vehiculo = 0
    peso_actual = 0
    print(f"\n🚛 Vehículo {v}:")
    for i in [almacen] + clientes:
        for j in clientes + [almacen]:
            if (v, i, j) in x and x[v, i, j].varValue == 1:
                distancia = df_distance.at[i, j]
                costo_km = df_vehicle[df_vehicle["vehiculo_id"] == v]["costo_km"].values[0]
                costo_vehiculo += distancia * costo_km
                if i == almacen:
                    print(f"   - 🚛 Reabastecimiento en {almacen}")
                    peso_actual = 0  # Reiniciar el peso al recargar
                else:
                    pedido_cliente = df_orders[df_orders["cliente"] == i]["order_demand"].values[0]
                    if peso_actual + pedido_cliente > df_vehicle[df_vehicle["vehiculo_id"] == v]["capacidad_kg"].values[0]:
                        print(f"   - 🚛 Reabastecimiento en {almacen} antes de ir a {i}")
                        peso_actual = 0  # Reiniciar el peso
                    peso_actual += pedido_cliente
                print(f"   - De {i} a {j}: {distancia} km, Costo: ${distancia * costo_km:.2f}")
    capacidad = df_vehicle[df_vehicle["vehiculo_id"] == v]["capacidad_kg"].values[0]
    print(f"   🔹 Peso total transportado en este viaje: {peso_actual} kg / Capacidad: {capacidad} kg")
    print(f"   🔹 Costo total del vehículo: ${costo_vehiculo:.2f}")
    costo_total += costo_vehiculo

print("\n💰 Costo total de todas las rutas: ${:.2f}".format(costo_total))



Resumen de rutas:

🚛 Vehículo 5:
   - 🚛 Reabastecimiento en Almacén
   - De Almacén a Cliente_9: 1.0494 km, Costo: $0.34
   - De Cliente_1 a Cliente_6: 1.7407 km, Costo: $0.56
   - De Cliente_2 a Cliente_3: 3.3838 km, Costo: $1.08
   - De Cliente_3 a Cliente_19: 7.7574 km, Costo: $2.48
   - De Cliente_4 a Cliente_1: 1.1998 km, Costo: $0.38
   - De Cliente_6 a Almacén: 2.6952 km, Costo: $0.86
   - De Cliente_7 a Cliente_16: 7.219399999999999 km, Costo: $2.31
   - De Cliente_9 a Cliente_13: 7.722899999999999 km, Costo: $2.47
   - De Cliente_13 a Cliente_2: 2.6961 km, Costo: $0.86
   - De Cliente_16 a Cliente_4: 1.3127 km, Costo: $0.42
   - De Cliente_19 a Cliente_7: 9.828899999999999 km, Costo: $3.15
   🔹 Peso total transportado en este viaje: 9268 kg / Capacidad: 10000 kg
   🔹 Costo total del vehículo: $14.91

🚛 Vehículo 6:
   - 🚛 Reabastecimiento en Almacén
   - De Almacén a Cliente_10: 5.9516 km, Costo: $0.83
   - De Cliente_10 a Cliente_12: 2.3061 km, Costo: $0.32
   - De Cliente_12